In [1]:
import numpy as np

In [1]:
from __future__ import print_function
import pints
import pints.toy as toy
import pints._diagnostics as diagnostics
import numpy as np
import matplotlib.pyplot as pl

# Load a forward model
model = toy.LogisticModel()

# Create some toy data
real_parameters = [0.015, 500]
times = np.linspace(0, 1000, 1000)
org_values = model.simulate(real_parameters, times)

# Add noise
noise = 10
values = org_values + np.random.normal(0, noise, org_values.shape)
real_parameters = np.array(real_parameters + [noise])

# Get properties of the noise sample
noise_sample_mean = np.mean(values - org_values)
noise_sample_std = np.std(values - org_values)

# Create an object with links to the model and time series
problem = pints.SingleSeriesProblem(model, times, values)

# Create a log-likelihood function (adds an extra parameter!)
log_likelihood = pints.UnknownNoiseLogLikelihood(problem)

# Create a uniform prior over both the parameters and the new noise variable
prior = pints.UniformPrior(
    [0.01, 400, noise*0.1],
    [0.02, 600, noise*100]
    )

# Create a Bayesian log-likelihood (prior * likelihood)
log_likelihood = pints.LogPosterior(prior, log_likelihood)

# Create an adaptive covariance MCMC routine
x0 = real_parameters * 1.1
mcmc = pints.AdaptiveCovarianceMCMC(log_likelihood, x0)

# Aim for an acceptance rate of 0.3
mcmc.set_acceptance_rate(0.3)

# Use 4000 iterations in total
mcmc.set_iterations(500)

# Start adapting after 1000 iterations
mcmc.set_non_adaptive_iterations(500)

# Discard the first 2000 iterations as burn in
mcmc.set_burn_in(250)

# Store only every 4th sample
mcmc.set_thinning_rate(1)

# Disable verbose mode
mcmc.set_verbose(False)

# Number of MCMC chains to run
nChains = 4

# Run!
print('Running...')
for i in range(0,nChains):
    print(i)
    chain = mcmc.run()
    if i == 0:
        chains = [chain]
    else:
        chains.append(chain)
print('Done!')

Running...
0
1
2
3
Done!


In [2]:
chains[0].shape

(250, 3)

In [3]:
def within(samples):
    mu = map(lambda x: np.var(x), samples)
    W = np.mean(mu)
    return W
def between(samples):
    mu = map(lambda x: np.mean(x), samples)
    mu_overall = np.mean(mu)
    m = len(samples)
    t = len(samples[0])
    return (t / (m - 1.0)) * np.sum((mu - mu_overall) ** 2)
def rhat(samples):
    W = within(samples)
    B = between(samples)
    t = len(samples[0])
    return np.sqrt((W + (1.0 / t) * (B - W)) / W)

In [4]:
samples = [chains[0][:,0],chains[1][:,0],chains[2][:,0],chains[0][:,0]]

In [5]:
diagnostics.rhat(samples)

1.0568056801593742